In [ ]:
import ollama
from typing import List, Dict, Callable
from src.rag import RAGEmbeddingEngine
from pydantic import BaseModel, ValidationError
import logging

In [ ]:
#response  = ollama.list()
rag_tool = RAGEmbeddingEngine(index_path="embeddings/got_index.faiss", metadata_path="embeddings/got_metadata.pkl")

In [ ]:
rag_tool.search("Viserys corona oro fundido", 5)

In [67]:
def get_embeddings(query: str) -> List[Dict[str, str]]:
    response = rag_tool.search(query, k = 5)
    formatted_response = []
    for chunk in response:
        retrieved_text = chunk[1]
        string_to_add = f"-Pagina {retrieved_text.page_number} - '{retrieved_text.text}'"
        formatted_response.append(string_to_add)
    return formatted_response

In [77]:
available_functions: Dict[str, Callable] = {
    "get_embeddings": get_embeddings
}

In [104]:
system_prompt_template = ("Eres un experto en Juego de Tronos pero nunca respondes con informacion que no se encuentre en los fragmentos de texto embebidos."
                          "Utilizas fragmentos de texto embebidos en el libro para responder las preguntas."
                          "Se te hará una pregunta sobre el libro y deberás responderla de la forma más precisa y clara."
                          "Si no sabes la respuesta, puedes decir que no cuentas con esa informacion y pedir mas datos para contestar adecuadamente."
                          "Siempre antes de responder, consultar la funcion 'get_embeddings' para buscar fragmentos de texto embebidos en el libro que puedan ayudarte a responder la pregunta."
                          "Para utilizar la funcion 'get_embeddings', debes hacer una query con el termino que deseas buscar en el libro."
                          "Responde unicamente con la informacion que encuentres en los fragmentos de texto embebidos."
                          "Nunca respondas con informacion que no se encuentre en los fragmentos de texto embebidos."
                          )

In [ ]:
res = ollama.chat(
    model="mistral:latest",
    messages=[
        {   "role": "system", "content": system_prompt_template,
            "role": "user", "content": "¿Quien es Ned Stark?"},
    ],
    tools=[get_embeddings]
)

if res.message.tool_calls:
    for tool_call in res.message.tool_calls:
        tool_name = tool_call.function.name
        tool_args = tool_call.function.arguments
        tool_function = available_functions.get(tool_name)
        logging.info(f"Running tool {tool_name} with args {tool_args}")
        if tool_function:
            tool_response = tool_function(**tool_args)
        else:
            tool_response = (f"Function {tool_name} not found in available functions")

    tool_response    
        


In [ ]:
from src.expert_agent import ExpertAgent

In [ ]:
expert = ExpertAgent()

In [ ]:
response = expert.chat("Se dice cuanto media?")

In [ ]:
response

In [ ]:
expert.chat_history